In [3]:
from QUBO_grover import *

from docplex.mp.model import Model
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_optimization.translators import from_docplex_mp

In [7]:
from docplex.mp.model import Model
import numpy as np

import numpy as np
from docplex.mp.model import Model
from qiskit_optimization.translators import from_docplex_mp

def qubo_to_model(Q):
    """
    Given a QUBO matrix Q, returns a docplex model and the equivalent QuadraticProgram.
    
    Q must be a square numpy array (symmetric preferred).
    """
    n = Q.shape[0]
    model = Model()
    
    # Create binary variables
    x = [model.binary_var(name=f"x{i}") for i in range(n)]
    
    # Build objective: sum of linear + quadratic terms
    objective = 0
    for i in range(n):
        objective += Q[i, i] * x[i]  # linear terms
        for j in range(i+1, n):
            if Q[i, j] != 0:
                objective += Q[i, j] * x[i] * x[j]  # quadratic terms
    
    model.minimize(objective)
    
    # Convert to QuadraticProgram
    qp = from_docplex_mp(model)
    return model, qp

# ---------------- Example ----------------
Q = np.array([[ 1,  2, -5],
              [ 2,  1,  0],
              [-5, 0,  3]])

model, qp = qubo_to_model(Q)
print(qp.prettyprint())


Problem name: docplex_model5

Minimize
  2*x0*x1 - 5*x0*x2 + x0 + x1 + 3*x2

Subject to
  No constraints

  Binary variables (3)
    x0 x1 x2



In [8]:
model, qp = qubo_to_model(Q)
print(qp.prettyprint())


grover_optimizer = GroverOptimizer(3, num_iterations=10, sampler=StatevectorSampler(seed=123))
results = grover_optimizer.solve(qp)
print(results.prettyprint())

Problem name: docplex_model6

Minimize
  2*x0*x1 - 5*x0*x2 + x0 + x1 + 3*x2

Subject to
  No constraints

  Binary variables (3)
    x0 x1 x2

objective function value: 4.0
variable values: x0=0.0, x1=1.0, x2=1.0
status: SUCCESS
